# Editing Headers of FITS Images #
We need to update the headers in some of our images because the TCS system was not communicating reliably while we were taking data.


References:
* http://www.astropy.org/astropy-tutorials/FITS-header.html
* http://docs.astropy.org/en/stable/io/fits/

In [3]:
from astropy.io import fits
from matplotlib import pyplot as plt
%matplotlib inline
import numpy as np

In [4]:
ref_image = 'images/r1442801.fit'
bad_image = 'images/r1442802.fit'

## Option 1 ##
Copy entire primary header of reference image to the bad image.

In [31]:
## this swaps the entire header
hdu1 = fits.open(ref_image)
ref_header = hdu1[0].header

hdu2 = fits.open(bad_image)
hdu2[0].header = hdu1[0].header
hdu2.writeto('images/fixed_image.fits',overwrite=True)
hdu1.close()
hdu2.close()

## Option 2 ##
Swap only a few fields

In [32]:
## swap only a few fields
header_keywords = ['LATITUDE','LONGITUDE','HEIGHT','SLATEL','TELESCOP','TELSTAT','RA', 'DEC','EQUINOX','RADECSYS']
hdu1 = fits.open(ref_image)
ref_header = hdu1[0].header
hdu2 = fits.open(bad_image)
for kw in header_keywords:
    hdu2[0].header[kw] = hdu1[0].header[kw]
hdu2.writeto('images/fixed_image.v2.fits',overwrite=True)
hdu1.close()
hdu2.close()

## The Winner !!!  ##

In [40]:
## hybrid approach
# read in files
hdu1 = fits.open(ref_image)
ref_header = hdu1[0].header
hdu2 = fits.open(bad_image)

# copy header of bad image to a temp location
bad_header = hdu2[0].header
# copy header of good image to the bad image
hdu2[0].header = hdu1[0].header

# go through orginal header of bad image
# replace values in copied header with original values from bad image
good_keywords = list(bad_header.keys())
for kw in good_keywords:
    if kw == 'COMMENT':
        continue
    hdu2[0].header[kw] = bad_header[kw]

# write image with fixed header    
hdu2.writeto('images/fixed_image.v3.fits',overwrite=True)
# close images
hdu1.close()
hdu2.close()